In [1]:
import pandas as pd
import numpy as np

In [2]:
root = '../HW_Datasets/movie_lens/'
movies_path = 'movies.dat'
ratings_path = 'ratings.dat'

In [3]:
def get_df(path, delim, columns):
    l = []

    with open(path) as file:
        for i in file:
            l.append(i.strip().split(delim))
            
    n_cols = range(len(i.split(delim)))
    return pd.DataFrame(l).rename(columns={old:new for old, new in zip(n_cols, columns)})

In [4]:
ratings = get_df(root+ratings_path, '::', 'userId,movieId,rating,timestamp'.split(','))

# movies = get_df(root+movies_path, '::', 'movieId,title,genres'.split(','))

In [5]:
# ratings = pd.read_csv(root+ratings_path)

# movies = pd.read_csv(root+movies_path)

In [6]:
ratings.head()

,userId,movieId,rating,timestamp
0,1,1193,5,978300760
1,1,661,3,978302109
2,1,914,3,978301968
3,1,3408,4,978300275
4,1,2355,5,978824291


# Surprise library

In [7]:
from surprise import KNNWithMeans, SVD, CoClustering
from surprise import Dataset
from surprise import accuracy
from surprise import Reader
from surprise.model_selection import train_test_split, cross_validate

### Prepare data

In [8]:
data = pd.DataFrame({
    'uid' : ratings.userId,
    'iid' : ratings.movieId,
    'rating' : ratings.rating
})

In [9]:
data.head()

,uid,iid,rating
0,1,1193,5
1,1,661,3
2,1,914,3
3,1,3408,4
4,1,2355,5


In [10]:
min_rating, max_rating = map(int, (ratings.rating.min(), data.rating.max()))

In [11]:
reader = Reader(rating_scale=(min_rating, max_rating))
dataframe = Dataset.load_from_df(data, reader)

## Baseline model evaluating

In [12]:
def validate_model(model_init, data):
    scores = cross_validate(model_init, 
                   data, 
                   measures=['rmse'], 
                   cv=5)['test_rmse']
    for ind in range(len(scores)):
        print(f'Fold # {ind+1}\nrmse: {scores[ind]}')

### user-based KNN_Zscore

In [13]:
validate_model(KNNWithMeans(sim_options={'user_based':True}, verbose=False), dataframe)

Fold # 1
rmse: 0.9301522761169128
Fold # 2
rmse: 0.9283096332701477
Fold # 3
rmse: 0.9309087715527145
Fold # 4
rmse: 0.9293538967345186
Fold # 5
rmse: 0.9271354879934354


### item-based KNN

In [14]:
validate_model(KNNWithMeans(sim_options={'user_based':False}, verbose=False), dataframe)

Fold # 1
rmse: 0.8833479597504595
Fold # 2
rmse: 0.8865681561170657
Fold # 3
rmse: 0.8872016113880883
Fold # 4
rmse: 0.8857196407673283
Fold # 5
rmse: 0.8856417731062128


### Also trying SVD

In [21]:
validate_model(SVD(n_factors=50, n_epochs=20, lr_all=0.005, random_state=5), dataframe)

Fold # 1
rmse: 0.8710051056746314
Fold # 2
rmse: 0.8710079706041745
Fold # 3
rmse: 0.868693663653425
Fold # 4
rmse: 0.8682525964639751
Fold # 5
rmse: 0.8714838809358886


### And also CoClustering

In [17]:
validate_model(CoClustering(random_state=5, verbose=False), dataframe)

Fold # 1
rmse: 0.9139672151179101
Fold # 2
rmse: 0.9171970387820558
Fold # 3
rmse: 0.91484533913582
Fold # 4
rmse: 0.9152173742355871
Fold # 5
rmse: 0.9141413938437719


# Conclusion

1) item-based, user-based KNN didnt perform so well
3) CoClustering is some kind of <s>bullshit</s> model that doesnt fit our task or maybe data
4) SVD is our saviour. For this task has always been the best performer considering learninig time and performance